# 🎵 HeartMuLa: 音楽生成推論

このノートブックでは、最先端のオープンソース音楽生成モデル **HeartMuLa** (Heart Music Language) を実行できます。

 <a href="https://github.com/HeartMuLa/heartlib">GitHub リポジトリ</a> | <a href="https://arxiv.org/abs/2601.10547">ArXiv 論文</a>

### ⚠️ ランタイム要件
**GPU が必要です。** GPU ランタイムで実行していることを確認してください。
*   **Colab:** ランタイム > ランタイムのタイプを変更 > T4 GPU (Standard) または A100 (Premium)。
*   **注意:** T4 GPU はメモリが限られています。VRAM 使用量を最小限に抑えるために `lazy_load=True` を使用します。

## 1. 環境セットアップ

In [1]:
# @title 依存関係のインストール
# @markdown リポジトリをクローンし、必要なPythonパッケージをインストールします。

import os
from IPython.display import clear_output

!nvidia-smi

print("依存関係をインストール中... (1〜2分かかります)")

# リポジトリのクローン
if not os.path.exists("/content/heartlib"):
    %cd /content
    !git clone https://github.com/HeartMuLa/heartlib.git

%cd /content/heartlib

# 依存関係のインストール
!pip install . --quiet
!pip install huggingface_hub --quiet

clear_output()
print("✅ インストール完了")

✅ インストール完了


## 2. モデルのダウンロード
Hugging Face から以下の3つのコンポーネントをダウンロードする必要があります：
1.  **設定とトークナイザー**: `HeartMuLa/HeartMuLaGen`
2.  **音楽モデル (3B)**: `HeartMuLa/HeartMuLa-oss-3B`
3.  **オーディオコーデック**: `HeartMuLa/HeartCodec-oss`

In [2]:
# @title チェックポイントのダウンロード

import os

# パスの定義
ckpt_dir = "/content/heartlib/ckpt"
os.makedirs(ckpt_dir, exist_ok=True)

print("設定とトークナイザーをダウンロード中...")
!huggingface-cli download HeartMuLa/HeartMuLaGen --local-dir {ckpt_dir} --local-dir-use-symlinks False

print("HeartMuLa-oss-3B モデルをダウンロード中...")
!huggingface-cli download HeartMuLa/HeartMuLa-oss-3B --local-dir {ckpt_dir}/HeartMuLa-oss-3B --local-dir-use-symlinks False

print("HeartCodec-oss をダウンロード中...")
!huggingface-cli download HeartMuLa/HeartCodec-oss --local-dir {ckpt_dir}/HeartCodec-oss --local-dir-use-symlinks False

clear_output()
print("✅ モデルが ./ckpt に正常にダウンロードされました")

✅ モデルが ./ckpt に正常にダウンロードされました


## 3. モデルパイプラインの読み込み
ここでパイプラインを初期化します。GPUが `bfloat16` (Ampere/A100) をサポートしているか、それとも `float16` / `float32` (T4/Pascal) にフォールバックすべきかを自動検出します。

In [3]:
import torch
from heartlib import HeartMuLaGenPipeline
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available() and torch.cuda.is_bf16_supported():
    mula_dtype = torch.bfloat16
    print("bfloat16 精度を使用します (推奨)")
else:
    mula_dtype = torch.float16
    print("float16 精度を使用します (T4/V100 向け)")
codec_dtype = torch.float32
pipe = HeartMuLaGenPipeline.from_pretrained(
    "./ckpt",
    device={"mula": device, "codec": device},
    dtype={"mula": mula_dtype, "codec": codec_dtype},
    version="3B",
    lazy_load=True
)
print("✅ パイプラインが正常に読み込まれました！")

bfloat16 精度を使用します (推奨)
Model components will be loaded to devices as specified:
  mula: cuda
  codec: cuda
✅ パイプラインが正常に読み込まれました！


## 4. 音楽生成

以下に歌詞とタグを入力してください。
*   **Lyrics**: `[Verse]`, `[Chorus]` のようなタグで構成してください。
*   **Tags**: ジャンルやムードをカンマ区切りで入力 (例: `pop, happy, piano`)。
*   **Parameters**:
    *   `Max Duration`: 曲の長さ（秒） (1分 = 60000ms)。
    *   `CFG Scale`: 値が高いほどテキストに忠実になります (標準は 1.5)。
    *   `Temperature`: 創造性 (標準は 1.0)。

In [6]:
# @title 🎹 推論設定

lyrics = """[Intro]
Stuck in my head
You’re stuck in my head
Like—

[Chorus]
You’re like Velcro in my brain
I shake you off
You stay
Every little word you say
Keeps looping
Looping
I try to press erase
But you’re printed on my days
You’re Velcro in my brain
I’m hooked and I’m not moving (hey)

[Verse 1]
Hit me once
I’m humming all week
Name on my tongue
Can’t speak
Scroll
Scroll
Still see your face
Every app
Same chase
Same taste
You’re the line that I can’t unsing
Wrong time
But the right feeling
Why you living in my mind like rent’s free
Why you always right here with me

[Chorus]
You’re like Velcro in my brain
I shake you off
You stay
Every little word you say
Keeps looping
Looping
I try to press erase
But you’re printed on my days
You’re Velcro in my brain
I’m hooked and I’m not moving (ooh)

[Bridge]
Pull you off
You just snap back
Heart off-track
Still run it back
I don’t wanna
But I press replay
Same old song
Brand new day

[Chorus]
You’re like Velcro in my brain
I shake you off
You stay
Every little word you say
Keeps looping
Looping
I try to press erase
But you’re printed on my days
You’re Velcro in my brain
Guess I kinda like this feeling (yeah)"""

tags = "Retro wave, Synth Wave, Female Vocals" # @param {type:"string"}

duration_seconds = 240 # @param {type:"slider", min:10, max:180, step:10}
guidance_scale = 3.1 # @param {type:"slider", min:1.0, max:5.0, step:0.1}
temperature = 1.0 # @param {type:"slider", min:0.1, max:2.0, step:0.1}

# 長さをミリ秒に変換
max_audio_length_ms = duration_seconds * 1000

output_path = "./assets/my_generation.mp3"
os.makedirs("./assets", exist_ok=True)

print("音楽を生成中... お待ちください。")

with torch.no_grad():
    pipe(
        {
            "lyrics": lyrics,
            "tags": tags,
        },
        max_audio_length_ms=max_audio_length_ms,
        save_path=output_path,
        topk=50,
        temperature=temperature,
        cfg_scale=guidance_scale,
    )

print(f"✅ 生成完了: {output_path}")

音楽を生成中... お待ちください。


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

 50%|█████     | 1505/3000 [09:53<09:49,  2.53it/s]


You have set lazy_load=True. Unloading HeartMuLa from device.
CUDA memory before unloading: 12.67 GB
CUDA memory after unloading: 0.01 GB


100%|██████████| 10/10 [00:13<00:00,  1.33s/it]


You have set lazy_load=True. Unloading HeartCodec from device.
CUDA memory before unloading: 6.20 GB
CUDA memory after unloading: 0.01 GB
✅ 生成完了: ./assets/my_generation.mp3


In [7]:
# @title 🎧 生成した曲を試聴
from IPython.display import Audio, display

if os.path.exists(output_path):
    display(Audio(output_path))
else:
    print("音声ファイルが見つかりません。先に生成セルを実行してください。")

In [ ]:
# @title (オプション) MP3のダウンロード
# @markdown 生成されたファイルをローカルコンピュータにダウンロードするにはこれを実行してください。
from google.colab import files
if os.path.exists(output_path):
    files.download(output_path)